# NeBender chatbot using Seq2Seq LSTM models
This notebook is a fork of the - https://colab.research.google.com/github/sindar/test-notebooks/blob/master/ChatBot_With_Seq2Seq.ipynb

In this notebook, we will assemble a seq2seq LSTM model using Keras Functional API to create a working chatbot trained on the dataset based on dialogues of the robor Bender from Futurama.


## 1) Importing the packages

We will import [TensorFlow](https://www.tensorflow.org) and our beloved [Keras](https://www.tensorflow.org/guide/keras). Also, we import other modules which help in defining model layers.






In [1]:

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing


## 2) Preprocessing the data

### A) Reading the data from the files

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.





In [2]:

from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = 'bender_nlp/episodes'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

# with open("questions.txt", "w") as file:
#     # file.writelines(questions)
#     for question in questions:
#         file.write(question + '\n')


answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 4508



### B) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum length.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum length.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.



In [3]:

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

#p = tokenize( questions + answers )
#model = Word2Vec( p[ 0 ] ) 

#embedding_matrix = np.zeros( ( VOCAB_SIZE , 100 ) )
#for i in range( len( tokenizer.word_index ) ):
    #embedding_matrix[ i ] = model[ vocab[i] ]

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(1299, 48) 48
(1299, 73) 73
(1299, 73, 4508)


## 3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.



<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).










In [4]:

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 73)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 48, 200)      901600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 73, 200)      901600      input_2[0][0]                    
______________________________________________________________________________________________

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [13]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=64, epochs=100 )
model.save( './model/model.h5' )

model_structure = model.to_json()
with open("./model/model_struct.json", "w") as json_file:
    json_file.write(model_structure)


Epoch 1/100
21/21 [==============================] - 35s 1s/step - loss: 1.1171
Epoch 2/100
21/21 [==============================] - 71s 3s/step - loss: 0.9778
Epoch 3/100
21/21 [==============================] - 71s 3s/step - loss: 0.9477
Epoch 4/100
21/21 [==============================] - 70s 3s/step - loss: 0.9236
Epoch 5/100
21/21 [==============================] - 69s 3s/step - loss: 0.9073
Epoch 6/100
21/21 [==============================] - 27s 1s/step - loss: 0.8953
Epoch 7/100
21/21 [==============================] - 47s 2s/step - loss: 0.8855
Epoch 8/100
21/21 [==============================] - 66s 3s/step - loss: 0.8761
Epoch 9/100
21/21 [==============================] - 67s 3s/step - loss: 0.8674
Epoch 10/100
21/21 [==============================] - 68s 3s/step - loss: 0.8586
Epoch 11/100
21/21 [==============================] - 67s 3s/step - loss: 0.8499
Epoch 12/100
21/21 [==============================] - 68s 3s/step - loss: 0.8414
Epoch 13/100
21/21 [=================

### 4.1 Loading the pre-trained model


In [5]:
# from keras.models import model_from_json
# with open('model_struct.json', 'r') as f:
#     model_json = f.read()

# model = model_from_json(model_json)

model.load_weights('model.h5')

## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [6]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


## 6) Talking with our Chatbot

First, we define a method `str_to_tokens` which converts `str` questions to Integer tokens with padding.


In [7]:

def str_to_tokens( sentence : str ):
    # sentence = re.sub(r'[^\w]', ' ', sentence)
    sentence = sentence.replace('?', ' ')
    sentence = sentence.replace('.', ' ')
    sentence = sentence.replace(',', ' ')
    sentence = sentence.replace('!', ' ')
    sentence = sentence.replace('-', ' ')
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append( tokenizer.word_index[ word ] ) 
        except:
            a = 1
            # print("=>token doesn't exist in the vocabulary")
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')





1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer length.







In [8]:

enc_model , dec_model = make_inference_models()

def infer_answer(question):
    states_values = enc_model.predict(str_to_tokens(question))
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    
    #remove last word if "end"
    dec_tr_list = decoded_translation.split()
    last_index = len(dec_tr_list) - 1
    if dec_tr_list[last_index] == 'end':
        dec_tr_list.pop()
        decoded_translation = " "
        decoded_translation = decoded_translation.join(dec_tr_list)
        # string2=" ".join(map(str,list1))

    return decoded_translation

for _ in range(4):
    answer = infer_answer(input( 'Enter question : '))
    print(answer)
    

hey fender man i haven't seen you since high school you still workin' at jack in the box
no thanks
no thanks
no thanks



## 7) Conversion to TFLite ( Optional )

We can convert our seq2seq model to a TensorFlow Lite model so that we can use it on edge devices.


In [21]:

!pip install tf-nightly


ERROR: Could not find a version that satisfies the requirement tf-nightly (from versions: none)
ERROR: No matching distribution found for tf-nightly


In [22]:

converter = tf.lite.TFLiteConverter.from_keras_model( enc_model )
buffer = converter.convert()
open( 'enc_model.tflite' , 'wb' ).write( buffer )

converter = tf.lite.TFLiteConverter.from_keras_model( dec_model )
open( 'dec_model.tflite' , 'wb' ).write( buffer )


INFO:tensorflow:Assets written to: /tmp/tmpq90q3z5u/assets


INFO:tensorflow:Assets written to: /tmp/tmpq90q3z5u/assets


6224596